### 1. 简介

- 输入数据是 `(x, y⁺, y⁻)`：同一个 prompt，两个回答，一个好一个坏。
- 目标：让 policy 模型在好回答上的 log-prob 高于坏回答。
- 和 RLHF 不同：DPO 不需要 reward model，只需要偏好对。

In [1]:
import torch
import torch.nn.functional as F

def dpo_loss(policy_logp_pos, policy_logp_neg,
             ref_logp_pos=None, ref_logp_neg=None,
             beta=0.1, reference_free=False):
    """
    Direct Preference Optimization (DPO) 损失函数
    ------------------------------------------------
    Args:
        policy_logp_pos: policy 模型在 "好回答 y⁺" 上的 log-prob (batch,)
        policy_logp_neg: policy 模型在 "坏回答 y⁻" 上的 log-prob (batch,)
        ref_logp_pos:    reference 模型在 "好回答 y⁺" 上的 log-prob (batch,)
        ref_logp_neg:    reference 模型在 "坏回答 y⁻" 上的 log-prob (batch,)
        beta:            温度系数 (常取 0.05 ~ 0.5)，控制损失的平滑程度
        reference_free:  若 True，则忽略 reference 模型（退化为无参对比学习）

    Returns:
        loss: 标量张量，batch 的平均 DPO 损失
    """

    # 1. policy 模型的 log-ratio：好回答 vs 坏回答
    #    log πθ(y⁺|x) - log πθ(y⁻|x)
    pi_logratio = policy_logp_pos - policy_logp_neg

    # 2. reference 模型的 log-ratio（如果不开启 reference_free）
    #    log π_ref(y⁺|x) - log π_ref(y⁻|x)
    ref_logratio = 0 if reference_free else (ref_logp_pos - ref_logp_neg)

    # 3. 最终对比 logits = policy 相对优势 - reference 相对优势
    #    这是 DPO 论文里的 h_{πθ}^{y⁺, y⁻}
    logits = pi_logratio - ref_logratio

    # 4. DPO 损失
    #    -log σ(β * logits)
    #    当 logits 越大（说明 policy 更倾向好回答），loss 越小
    loss = -F.logsigmoid(beta * logits).mean()

    return loss


In [2]:
# ---------------------------
# Case 0: 基础设置与直觉
# ---------------------------
# 构造一个 batch，policy 对正样本更“自信”，负样本更“不自信”
# logp 通常为负数；数值越接近 0 表示概率越大
policy_logp_pos = torch.tensor([-1.0, -0.2, -2.0, -0.5])  # 好回答 y+
policy_logp_neg = torch.tensor([-2.0, -1.5, -3.0, -1.0])  # 坏回答 y-

# 参考模型（ref）与 policy 差不多或者更弱
ref_logp_pos = torch.tensor([-1.2, -0.5, -2.5, -0.7])
ref_logp_neg = torch.tensor([-1.8, -1.3, -2.7, -0.9])

# ---------------------------
# Case 1: reference_free = True
# 期望：loss1 较小（policy 确实更偏向 y+）
# ---------------------------
loss1 = dpo_loss(policy_logp_pos, policy_logp_neg, beta=0.1, reference_free=True)
print("Case 1 (reference_free=True) loss:", float(loss1))

# ---------------------------
# Case 2: 有 reference，且 policy 与 reference 相近
# 由于两者优势差不多，logits 变小 → 损失略变大
# ---------------------------
loss2 = dpo_loss(policy_logp_pos, policy_logp_neg,
                 ref_logp_pos, ref_logp_neg, beta=0.1, reference_free=False)
print("Case 2 (with reference, similar preference) loss:", float(loss2))

# ---------------------------
# Case 3: 构造“困难模式”
# reference 更偏向负样本（或更不偏向正样本），
# 这会放大 policy 相对 reference 的优势 → 损失更小
# ---------------------------
ref_bad_pos = torch.tensor([-3.0, -2.5, -3.5, -2.0])  # ref 认为 y+ 很差
ref_bad_neg = torch.tensor([-1.0, -0.9, -1.1, -0.8])  # ref 认为 y- 还不错
loss3 = dpo_loss(policy_logp_pos, policy_logp_neg,
                 ref_bad_pos, ref_bad_neg, beta=0.1, reference_free=False)
print("Case 3 (with reference, opposite preference) loss:", float(loss3))

Case 1 (reference_free=True) loss: 0.6468777656555176
Case 2 (with reference, similar preference) loss: 0.6685033440589905
Case 3 (with reference, opposite preference) loss: 0.5655654072761536


### 4. 怎么得到 `log π(y|x)`？

- 模型输出是 `[B, T, V]` 的 logits。
- 对齐 `logits[:, :-1]` 和 `labels[:, 1:]`（next-token 预测）。
- 用 `labels == -100` mask 掉 prompt 部分，只累计回答部分的 log-prob。
- `sum` 得到整条回答的 log-prob。